In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import openai
client = openai.OpenAI()

# 일반 프롬프트 vs Zero-shot CoT

In [4]:
# 일반 프롬프트
prompt = '123 x 31은 얼마인가요?'

# Zero shot CoT 적용 프롬프트
cot_prompt = '단계별로 생각해보세요: 123 x 31은 얼마인가요?'

res = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role':'user', 'content':prompt}]
)

cot_res = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role':'user', 'content':cot_prompt}]
)

In [5]:
res

ChatCompletion(id='chatcmpl-BARf03hX6prEDCcZeUerHFyxzGe6c', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='123 x 31 = 3813', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1741829306, model='gpt-4-0613', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=20, total_tokens=29, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [7]:
cot_res

ChatCompletion(id='chatcmpl-BARf1JUyqSQ6uPrdXjJho1AqwnY0v', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='123 x 31을 계산하기 위해선 아래와 같은 단계를 거칩니다.\n\n단계 1: 123 에 1를 곱하세요. -> 123\n단계 2: 123 에 30 을 곱하세요. -> 3690\n단계 3: 단계 1에서 계산한 123과 단계 2에서 계산한 3690을 합하세요. -> 3813\n\n따라서, 123 x 31 = 3813입니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1741829307, model='gpt-4-0613', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=131, prompt_tokens=35, total_tokens=166, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### Few-shot CoT

In [11]:
few_shot_cot = '''
123 x 31을 단계별로 계산해 보겠습니다.

1. 123를 분해합니다: ( 123 = 100 + 20 + 3 )
2. 각 항을 31과 곱합니다:
( 100 \times 31 = 3100 )
( 20 \times 31 = 620 )
( 3 \times 31 = 93 )

3. 이제 이 결과들을 더합니다:
( 3100 + 620 + 93 )

4. 계산을 진행하면:
( 3100 + 620 = 3720 )
( 3720 + 93 = 3813 )

따라서, ( 123 \times 31 = 3813 )입니다.

Q: 789 x 456 은 얼마인가요?
A: 
'''

res = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role':'user', 'content':few_shot_cot}]
)

In [12]:
print(res.choices[0].message.content)

1. 789를 분해합니다: (789 = 700 + 80 + 9)
2. 각 항을 456과 곱합니다:
(700 * 456 = 319200)
(80 * 456 = 36480)
(9 * 456 = 4104)
3. 이제 이 결과들을 더합니다:
(319200 + 36480 + 4104)
4. 계산을 진행하면:
(319200 + 36480 = 355680)
(355680 + 4104 = 359784)

따라서, (789 * 456 = 359784)입니다.


### 논리 문제 해결

In [13]:
prompt = "앨리스(Alice)는 밥(Bob)보다 나이가 많다. 밥(Bob)은 찰리(Charlie)보다 나이가 많다. 데이비드(David)는 앨리스보다 어리지만, 에밀리(Emily)보다는 많다. 찰리(Charlie)는 프랭크(Frank)보다 많다. 에밀리(Emily)는 찰리보다 나이가 많지만, 밥보다는 어리다. 프랭크(Frank)는 데이비드보다 어리다."
cot_prompt = "단계적으로 생각해서 대답해주세요: 앨리스(Alice)는 밥(Bob)보다 나이가 많다. 밥(Bob)은 찰리(Charlie)보다 나이가 많다. 데이비드(David)는 앨리스보다 어리지만, 에밀리(Emily)보다는 많다. 찰리(Charlie)는 프랭크(Frank)보다 많다. 에밀리(Emily)는 찰리보다 나이가 많지만, 밥보다는 어리다. 프랭크(Frank)는 데이비드보다 어리다."

In [14]:
res = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role':'user', 'content':prompt}]
)

cot_res = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role':'user', 'content':cot_prompt}]
)

In [ ]:
print(res.choices[0].message.content)

In [ ]:
print(cot_res.choices[0].message.content)

### 코드 디버깅

In [18]:
prompt = """
다음과 같이 코드를 작성했더니 에라가 발생합니다. 왜죠?

def add_numbers(a, b):
return a + b

print(add_numbers(1, 2))
"""

cot_prompt = """
다음과 같이 코드를 작성했더니 에라가 발생합니다.
실수가 무엇인지 찾아 단계별로 설명해 주세요.

def add_numbers(a, b):
return a + b

print(add_numbers(1, 2))
"""

In [ ]:
res = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role':'user', 'content':prompt}]
)

cot_res = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role':'user', 'content':cot_prompt}]
)

In [ ]:
print(cot_res.choices[0].message.content)